In [1]:
import sys
sys.path.insert(0, r'../..')

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import imageio
import pickle
import os
from tqdm import tqdm_notebook as tqdm

In [3]:
from lib.video import *
from lib.cross_val import *
from lib.utils import convert_time

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut, train_test_split
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [5]:
MODEL_DIR = '../../output/features/image_32_16/'
Y_DATA = '../../data/train/events_data.csv'
TRAIN_FILES = np.array(['641579_3.mp4','643734_5.mp4','633012_5.mp4','631638_5.mp4','631646_5.mp4','631750_5.mp4'])
EVENTS_TYPE = ['удар по воротам', 'угловой', 'замена', 'желтая карточка', 'гол']

In [6]:
MIN_COMBO = 4

In [7]:
y_data = pd.read_csv(Y_DATA)
y_data['event_time'] = y_data['event_time'].apply(convert_time)

In [8]:
def sampling(Xv, yv, k=1.0, max_size=13000):
    nX, ny = [], []
    bad_pairs = []
    for X, y in zip(Xv, yv):
        if y in EVENTS_TYPE:
            nX.append(X)
            ny.append(y)
        else:
            bad_pairs.append((X, y))
    bad_count = len(bad_pairs)
    for i in np.random.choice(bad_count, size=int(len(nX) * k)):
        X, y = bad_pairs[i]
        nX.append(X)
        ny.append(y)
    nX, ny = np.array(nX), np.array(ny)
    if max_size is not None and len(nX) > max_size:
        ind = np.random.choice(len(nX), size=max_size)
        nX, ny = nX[ind], ny[ind]
    return list(nX), list(ny)

In [9]:
def replace_none(y):
    return ['NONE' if v is None else v for v in y]

In [10]:
all_results = []

In [11]:
from collections import defaultdict

In [21]:
def load_Xy(files, is_sampling=True):
    X, y = [], []
    for file in files:
        path = MODEL_DIR + file + '.pickle'
        with open(path, 'rb') as f:
            Xv, yv, X_time = pickle.load(f)
        yv = replace_none(yv)
        if is_sampling:
            Xv, yv = sampling(Xv, yv)
        X += Xv
        y += yv
    X = np.array(X).reshape((len(X), -1))
    y = np.array(y)
    return X, y

def load_X(files, k=1):
    X, X_file_names, X_time = [], [], []
    for file in files:
        path = MODEL_DIR + file + '.pickle'
        with open(path, 'rb') as f:
            Xv, yv, X_time = pickle.load(f)
        X += Xv[::k]
        X_time += X_time[::k]
        X_file_names += [file] * len(Xv[::k])
    X = np.array(X).reshape((len(X), -1))
    return X, X_file_names, X_time

class MetaModel():
    def __init__(self, model):
        self.model = model
        self.pred_buf = {}
        self.fit_set = set()
    
    def fit(self, files):
        files = frozenset(files)
        if files in self.fit_set:
            return
        self.fit_set.add(files)
        X, y = load_Xy(files)
        print(y)
        print('Fit with', len(X), 'samples')
        self.model.fit(X, y)
        
    def score(self, files):
        X, y = load_Xy(files, False)
        print('Score with', len(X), 'samples')
        y_pred = self.model.predict(X)
        return f1_score(y, y_pred, average='macro')
        return pd.DataFrame({
            'true': y,
            'pred': y_pred
        })
        
        
    def predict(self, files):
        files = frozenset(files)
        X, X_file_names, X_time = load_X(files)
        print('Predict for', len(X), 'samples')
        if files in self.pred_buf:
            print('From pred_buf')
            y_pred = self.pred_buf[files]
        else:
            y_pred = self.model.predict(X)
            self.pred_buf[files] = y_pred
        result = []
        combo = 0
        combo_type = None
        last_event = defaultdict(lambda: -10000)
        for i in range(len(X)):
            if 0 <= X_time[i] <- 6200:
                continue
            if y_pred[i] in EVENTS_TYPE:
                if combo_type == y_pred[i]:
                    combo += 1
                else:
                    combo = 1
                combo_type = y_pred[i]
            else:
                combo = 0
            if combo == MIN_COMBO and last_event[combo_type] < X_time[i] - 59:
                result.append({
                    'file_name': X_file_names[i],
                    'event_type': combo_type,
                    'event_time': X_time[i]
                })
                last_event[combo_type] = X_time[i]
        result = pd.DataFrame(result)
        all_results.append(result)
        return result

In [22]:
def only_one_fold(cv):
    def fun(X):
        return [next(cv.split(X))]
    class s: pass
    obj = s
    s.split = fun
    return obj

In [23]:
ridge = MetaModel(RidgeClassifier(alpha=10, normalize=True, class_weight='balanced'))

In [24]:
ridge.fit(TRAIN_FILES[:2])
ans = ridge.score(TRAIN_FILES[-2:])
ans

['удар по воротам' 'удар по воротам' 'удар по воротам' ..., 'NONE' 'NONE'
 'NONE']
Fit with 22986 samples
Score with 26446 samples


0.16316893328899557

In [25]:
MIN_COMBO = 4
print(cross_val_score(ridge, y_data, TRAIN_FILES, LeaveOneOut()).mean())

A Jupyter Widget


['удар по воротам' 'удар по воротам' 'удар по воротам' ..., 'NONE' 'NONE'
 'NONE']
Fit with 58104 samples


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/josdas/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/josdas/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/josdas/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Predict for 7047 samples
task_score: true_positives=2 false_positives=1 false_negatives=19
task_score: empty y_pred
task_score: true_positives=4 false_positives=59 false_negatives=2
task_score: true_positives=0 false_positives=4 false_negatives=4
task_score: true_positives=0 false_positives=7 false_negatives=0
cross_val_score: score=0.030489510489510492 test_files=['641579_3.mp4']
['удар по воротам' 'удар по воротам' 'удар по воротам' ..., 'NONE' 'NONE'
 'NONE']
Fit with 58530 samples
Predict for 8714 samples
task_score: true_positives=2 false_positives=2 false_negatives=21
task_score: true_positives=0 false_positives=5 false_negatives=7
task_score: true_positives=3 false_positives=15 false_negatives=2
task_score: true_positives=4 false_positives=33 false_negatives=2
task_score: true_positives=0 false_positives=69 false_negatives=0
cross_val_score: score=0.06651282051282051 test_files=['643734_5.mp4']
['удар по воротам' 'удар по воротам' 'удар по воротам' ..., 'NONE' 'NONE'
 'NONE']
Fi

In [19]:
all_results[-1]

,event_time,event_type,file_name
0,-297,удар по воротам,631750_5.mp4
1,13,удар по воротам,631750_5.mp4
2,77,удар по воротам,631750_5.mp4
3,141,удар по воротам,631750_5.mp4
4,201,удар по воротам,631750_5.mp4
5,261,удар по воротам,631750_5.mp4
6,325,удар по воротам,631750_5.mp4
7,419,удар по воротам,631750_5.mp4
8,481,удар по воротам,631750_5.mp4
9,550,удар по воротам,631750_5.mp4
